## Data Cleaning

Let's load the some data for TSLA. Unfortunately, this data is not quite as *clean* as our NVDA data, so we'll need to do some data wrangling. The file we're looking to load is `TSLA_2015_2024.csv`.

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("data/TSLA_2015_2024.csv")
df.Date = pd.to_datetime(df.Date, format="%d/%m/%Y")

# set the index for making code cleaner and more efficient for date-related analysis.
df.set_index("Date", inplace=True) # apply the change directly to the DataFrame
df

,Close,High,Low,Open,Volume
Date,,,,,
2024-07-11,241.029999,271.000000,239.649994,263.299988,221707300.0
2020-08-13,108.066666,110.078667,104.484001,107.400002,306379500.0
2020-08-13,108.066666,110.078667,104.484001,107.400002,306379500.0
2019-10-30,21.000668,21.252666,20.664667,20.866667,144627000.0
2015-08-27,16.199333,NaN,15.387333,15.400000,114840000.0
...,...,...,...,...,...
2018-08-10,23.699333,24.000000,23.066668,23.600000,173280000.0
2021-06-21,206.943329,210.463333,202.960007,208.160004,74438100.0
2016-06-20,14.646667,14.916667,14.548667,14.633333,53332500.0


Can you see what we mean by messy? How many issues can you spot?

- Dates out of order
- Duplicate rows
- Missing values

## Ordering and Duplicates

First let's start with sorting the index.

In [6]:
df.index.is_monotonic_increasing # checks whether the index values are in non-decreasing order，helps you ensure your time series data is sorted by date

df.sort_index(inplace=True) # returns true if order is correct
df # but steps just sort of preview, not going to make changes to data frame if we not put (inplace=True)

df.index.is_monotonic_increasing

True

Now let's focus on duplicates:

In [7]:
df.duplicated().sum() # count all of the rows that were duplicated 
# in computer speak, the false = 0, and true = 1, this row got result of np.int64(50)

# Getting rid of duplicates 
df.drop_duplicates(inplace=(True))

# Check again if there any duplicates in data frame, 0 means no duplicates in data frame
df.duplicated().sum()

np.int64(0)

#### Tip: Method Chaining

**Method chaining** is a popular feature of pandas. It allows us to *chain* together several operations in a single line of code. For example, we can set the index, sort the data frame and drop any duplicates all at once. Notice we don't use `inplace` but rather re-assign to the original `df` variable.

```python
df = df.set_index("Date").sort_index().drop_duplicates()
```

## Not a Number (NaN)

### Exercise: Some Null Chain

Let's look at the missing or `NaN` values next. Previously, we saw that `info()` gave us some insight into how many missing values we had, but we can also use `isnull()`.

Can you chain `isnull()` with `sum()` to get a single value stating the total number of missing values in the data frame?

In [ ]:
## YOUR CODE GOES HERE
df.isnull() # tell us which rows which columns that are in 

df.isnull().sum() # this give us a total missing value within each column, not in total so far

df.isnull().sum().sum() # the total number of missing value in the whole entire data frame 

np.int64(101)

We can find out which rows have missing data using `isnull()`, `any()` along rows and some smart *masking*.

In [ ]:
df.isnull().any() # whether or not there's a missing value in anywhere in each column
condition = df.isnull().any(axis=1) # have the rows with missing value
df[condition]


,Close,High,Low,Open,Volume
Date,,,,,
2015-02-06,NaN,14.893333,14.433333,14.800000,48658500.0
2015-02-27,13.556000,13.903333,NaN,13.793333,58231500.0
2015-03-26,12.694000,12.986000,12.646667,NaN,61920000.0
2015-04-22,14.629333,NaN,14.112667,14.166667,117945000.0
2015-05-22,16.515333,16.573334,16.334000,NaN,33346500.0
...,...,...,...,...,...
2024-06-18,184.860001,NaN,182.369995,186.559998,68982300.0
2024-08-28,205.750000,211.839996,202.589996,209.720001,NaN
2024-10-18,220.699997,NaN,219.229996,220.710007,49611900.0


Now that we've identified our missing values, the big question is how to handle them. There are many approaches to this that will vary depending on the data and the further analysis you plan to carry out.

In [ ]:
df.dropna() #drop all na in database

df.dropna(subset="Close") # drop sepefic column's na, here is close, and missing value is still present in any other columm

df.Volume.fillna(df.Volume.mean()) # use the mean in volume column and fill it into missing value 

df.Close.interpolate(method="linear") 
# this will give back a filled close column where any missing value haven been interpolated by the two closest days on either side of it 

df.Close.ffill() # forward fill 
# take the price from the day before and just pop it into your day
# the idea behind is if missing a closed price from the day before, it's likely the market close that day, so just bring over the same day closed price 
# there is no change and it'll change the next day at the open

Date
2015-01-02     14.620667
2015-01-05     14.006000
2015-01-06     14.085333
2015-01-07     14.063333
2015-01-08     14.041333
                 ...    
2024-12-23    430.600006
2024-12-24    462.279999
2024-12-26    454.130005
2024-12-27    431.660004
2024-12-30    417.410004
Name: Close, Length: 2516, dtype: float64

### Exercise: Cleaning up

Notice how above we didn't actually update the `df` variable, so our DataFrame is still full of missing values. Fix all missing values applying the following rules:
- Fill missing Close by linear interpolation
- Fill missing Volume with the value from the day before
- Fill missing Open with the median Open
- Fill missing High with the Close or Open, whichever is higher
- Fill missing Low with a value 3% lower than the High


Your DataFrame `df` should have no missing values when done. Use `info()` to confirm.

**NOTE:** When changing values in a data frame, it is recommended to avoid using `inplace`, and instead re-assign the variable.

In [ ]:
## YOUR CODE GOES HERE
import pandas as pd

# Step 1: Fill missing Close with linear interpolation
df.Close = df.Close.interpolate(method="linear")

# Step 2: Fill missing Volume with forward fill (value from the day before)
df.Volume = df.Volume.ffill()

# Step 3: Fill missing Open with the median of the Open column
df.Open = df.Open.fillna(df.Open.median())

# Step 4: Fill missing High with the higher of Close or Open
df.High = df.High.fillna(df[["Close", "Open"]].max(axis=1)) # Across the columns （Row-wise) means gets the maximum value in each row.

# Step 5: Fill missing Low with 3% lower than the High
df.Low = df.Low.fillna(df.High * 0.97)

# Check the number of null number (should be 0)
df.isnull().sum().sum()

np.int64(0)

In [ ]:
# Final check
df.info()
df.Volume =df.Volume.astype("int64") # convert that volume back into an integer
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2516 entries, 2015-01-02 to 2024-12-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2516 non-null   float64
 1   High    2516 non-null   float64
 2   Low     2516 non-null   float64
 3   Open    2516 non-null   float64
 4   Volume  2516 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 117.9 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2516 entries, 2015-01-02 to 2024-12-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2516 non-null   float64
 1   High    2516 non-null   float64
 2   Low     2516 non-null   float64
 3   Open    2516 non-null   float64
 4   Volume  2516 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 117.9 KB


#### Advanced: Data Types

You may have noticed that the **Volume** column in the 2021 data frame is a `float64` instead of the `int64` *dtype* we had in the 2020 data frame. Missing values (NaN) are represented as a special case of floating point number, so all the values in **Volume** were automatically *upcast* to floats.

Ideally our columns should be of the *dtype* that most accurately represents them. This will improve performance when working with large data frames. Now that we've resolved our missing numbers, we can *cast* our trading volumes as integers.

In [ ]:
df.Volume =df.Volume.astype("int64") # convert that volume back into an integer
df.info()

## Saving Data

Now that we've cleaned our data, let's save it, by writing it to a new .CSV file. We can use pandas' `to_csv()`.

In [ ]:
df # check again if data is correct 
df.to_csv("TSLA_clean.csv")

In [11]:
df

,Close,High,Low,Open,Volume
Date,,,,,
2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000.0
2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500.0
2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500.0
2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000.0
2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500.0
...,...,...,...,...,...
2024-12-23,430.600006,434.510010,415.410004,NaN,72698100.0
2024-12-24,462.279999,462.779999,435.140015,435.899994,59551800.0
2024-12-26,454.130005,465.329987,451.019989,465.160004,76366400.0
